In [ ]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt
import matplotlib.animation as animation

dx = 1
Nx = 1000

e = np.ones(Nx)
diags = np.array([-1,0,1])
vals  = np.vstack((e,-2*e,e))
Lmtx = sp.spdiags(vals, diags, Nx, Nx);
Lmtx = sp.lil_matrix(Lmtx) # tranform mtx type to modify entries
Lmtx[0,Nx-1] = 1
Lmtx[Nx-1,0] = 1
Lmtx /= dx**2
Lmtx = sp.csr_matrix(Lmtx) # transform mtx type

In [ ]:
e = np.ones(Nx)
diags = np.array([-1,1])
vals  = np.vstack((-e,e))
Gmtx = sp.spdiags(vals, diags, Nx, Nx);
Gmtx = sp.lil_matrix(Gmtx) # tranform mtx type to modify entries
Gmtx[0,Nx-1] = -1
Gmtx[Nx-1,0] = 1
Gmtx /= (2*dx)
Gmtx = sp.csr_matrix(Gmtx) # transform mtx type

In [ ]:
def getAcc( pos, Nx, boxsize, n0, Gmtx, Lmtx ):
	N          = pos.shape[0]
	dx         = boxsize / Nx
	j          = np.floor(pos/dx).astype(int)
	jp1        = j+1
	weight_j   = ( jp1*dx - pos  )/dx
	weight_jp1 = ( pos    - j*dx )/dx
	jp1        = np.mod(jp1, Nx)   # periodic BC
	n  = np.bincount(j[:,0],   weights=weight_j[:,0],   minlength=Nx);
	n += np.bincount(jp1[:,0], weights=weight_jp1[:,0], minlength=Nx);
	n *= n0 * boxsize / N / dx 
	
	phi_grid = spsolve(Lmtx, n-n0)
	
	E_grid = - Gmtx @ phi_grid
	#plt.plot(E_grid)
	#plt.show()
	E = weight_j * E_grid[j] + weight_jp1 * E_grid[jp1]
	
	a = -E

	return a

In [ ]:
Nt = 100
dt = 0.01

N = 40000

pos = np.zeros((N,1))
pos[:int(N/2)] = np.random.uniform(0,1,(int(N/2),1))
pos[int(N/2):] = np.random.uniform(0,1,(int(N/2),1))

vel = np.zeros((N,1))
vel[:int(N/2)] = np.random.normal(0.1,0.01,(int(N/2),1))
vel[int(N/2):] = np.random.normal(-0.1,0.01,(int(N/2),1))


acc = np.zeros((N,1))
boxsize = 1
n0 = 45
t = 0

pos_m = np.zeros((Nt, N, 1))
vel_m = np.zeros((Nt, N, 1))
for i in range(Nt):
	vel += acc * dt/2.0
	pos += vel * dt
	pos = np.mod(pos, boxsize)
	acc = getAcc( pos, Nx, boxsize, n0, Gmtx, Lmtx )
	vel += acc * dt/2.0
	t += dt
 
	pos_m[i] = pos
	vel_m[i] = vel

	plt.scatter(pos[:int(N/2)], vel[:int(N/2)], s=0.4, c="red");plt.scatter(pos[int(N/2):], vel[int(N/2):], s=0.4, c="blue");plt.show()
	#plt.hist(pos, bins=100);plt.show()
	plt.xlim(0,1)

In [ ]:
fig = plt.figure()
ax = plt.axes(xlim=(0, 1))


line = ax.scatter(pos_m[i], vel_m[i], s=0.4)

def animate(i):
    line.set_offsets(np.c_[pos_m[i], vel_m[i]])
    return line,

anim = animation.FuncAnimation(fig, animate, frames=Nt, interval=1)
anim.save('plasma2.gif',writer='imagemagick') 

In [ ]:
def main():
	""" Plasma PIC simulation """
	
	# Simulation parameters
	N         = 40000   # Number of particles
	Nx        = 400     # Number of mesh cells
	t         = 0       # current time of the simulation
	tEnd      = 50      # time at which simulation ends
	dt        = 1       # timestep
	boxsize   = 50      # periodic domain [0,boxsize]
	n0        = 1       # electron number density
	vb        = 3       # beam velocity
	vth       = 1       # beam width
	A         = 0.1     # perturbation
	plotRealTime = True # switch on for plotting as the simulation goes along
	
	# Generate Initial Conditions
	np.random.seed(42)            # set the random number generator seed
	# construct 2 opposite-moving Guassian beams
	pos  = np.random.rand(N,1) * boxsize  
	vel  = vth * np.random.randn(N,1) + vb
	Nh = int(N/2)
	vel[Nh:] *= -1
	# add perturbation
	vel *= (1 + A*np.sin(2*np.pi*pos/boxsize))
	
	# Construct matrix G to computer Gradient  (1st derivative)
	dx = boxsize/Nx
	e = np.ones(Nx)
	diags = np.array([-1,1])
	vals  = np.vstack((-e,e))
	Gmtx = sp.spdiags(vals, diags, Nx, Nx);
	Gmtx = sp.lil_matrix(Gmtx)
	Gmtx[0,Nx-1] = -1
	Gmtx[Nx-1,0] = 1
	Gmtx /= (2*dx)
	Gmtx = sp.csr_matrix(Gmtx)

	# Construct matrix L to computer Laplacian (2nd derivative)
	diags = np.array([-1,0,1])
	vals  = np.vstack((e,-2*e,e))
	Lmtx = sp.spdiags(vals, diags, Nx, Nx);
	Lmtx = sp.lil_matrix(Lmtx)
	Lmtx[0,Nx-1] = 1
	Lmtx[Nx-1,0] = 1
	Lmtx /= dx**2
	Lmtx = sp.csr_matrix(Lmtx)
	
	# calculate initial gravitational accelerations
	acc = getAcc( pos, Nx, boxsize, n0, Gmtx, Lmtx )
	
	# number of timesteps
	Nt = int(np.ceil(tEnd/dt))
	
	# prep figure
	fig = plt.figure(figsize=(5,4), dpi=80)

	pos_m = np.zeros((Nt, N, 1))
	vel_m = np.zeros((Nt, N, 1))
	
	# Simulation Main Loop
	for i in range(Nt):
		# (1/2) kick
		vel += acc * dt/2.0
		
		# drift (and apply periodic boundary conditions)
		pos += vel * dt
		pos = np.mod(pos, boxsize)
		
		# update accelerations
		acc = getAcc( pos, Nx, boxsize, n0, Gmtx, Lmtx )
		
		# (1/2) kick
		vel += acc * dt/2.0
		
		# update time
		t += dt

		pos_m[i] = pos
		vel_m[i] = vel
		
		# plot in real time - color 1/2 particles blue, other half red
		if plotRealTime or (i == Nt-1):
			plt.cla()
			plt.scatter(pos[0:Nh],vel[0:Nh],s=.4,color='blue', alpha=0.5)
			plt.scatter(pos[Nh:], vel[Nh:], s=.4,color='red',  alpha=0.5)
			plt.axis([0,boxsize,-6,6])
			
			plt.pause(0.001)
			
	
	# Save figure
	plt.xlabel('x')
	plt.ylabel('v')
	plt.savefig('pic.png',dpi=240)
	plt.show()
	    
	return pos_m, vel_m
	

In [ ]:
pos_m, vel_m = main()

In [ ]:
%matplotlib tk
fig = plt.figure()
ax = plt.axes(xlim=(0, 1))
ax.axis([0,50,-6,6])

N = pos_m.shape[1]
Nt = pos_m.shape[0]


line1 = ax.scatter(pos_m[0,int(N/2):], vel_m[0,int(N/2):], s=0.4, c="blue")
line2 = ax.scatter(pos_m[0,:int(N/2)], vel_m[0,:int(N/2)], s=0.4, c="red")


def animate(i):
    line1.set_offsets(np.c_[pos_m[i,int(N/2):], vel_m[i,int(N/2):]])
    line2.set_offsets(np.c_[pos_m[i,:int(N/2)], vel_m[i,:int(N/2)]])
    return line1, line2,

anim = animation.FuncAnimation(fig, animate, frames=Nt, interval=50)
anim.save('plasma.gif',writer='imagemagick') 